In [1]:
import sys
import tensorflow as tf
import keras
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Reshape, Dense, Dropout, BatchNormalization, Concatenate
from keras import optimizers
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from keras import backend as K
from tqdm import tqdm


# reserve only the 30% of the GPU memory
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [19]:
batch_size = 512
width = 50
height = 10
channels = 3
classes = 6
epochs = 30
dataset_dir = '/lustrehome/adipilato/ParticleID/new_datasets/5PartPerEvent/padded/train/'
save_dir = '/lustrehome/adipilato/ParticleID/PID-ER_v2/models/'
padding = 'padding' + str(height)
model_name= padding + '_testTRT'
history_name = model_name + '_history'

class_labels = {22:0, -11:1, 11:1, -13:2, 13:2, -211:3, 211:3, 311:4, -1:5, -9999:5}

In [3]:
# # arrays of data needed for training

# data_list = []
# pid_list = []
# en_list = []

# # read dataset
# files = [f for f in os.listdir(dataset_dir) if f.endswith("h5")]

# for name in tqdm(files):
#     print("Reading file", name)
#     data = pd.read_hdf(dataset_dir + name)
#     num_tracks = data.trackster.max()
#     print(num_tracks)

#     tracksters = np.array([data.E.values, data.eta.values, data.phi.values]).T.reshape(-1, width, height, channels)

#     pid_vals = np.array([data.pid.values]).T.reshape(-1, width*height)
#     select = (np.max(pid_vals,axis=1)==0)
#     pid_vals=np.max(np.abs(pid_vals),axis=1)
#     pid_vals[select]=pid_vals[select]*-1.0
#     pid_vals = pid_vals.tolist()
#     pid_arr = [class_labels[x] for x in pid_vals]
#     en_vals = np.array([data.genE.values]).T.reshape(-1, width*height)
#     en_arr = np.max(en_vals,axis=1)

#     data_list.append(tracksters)
#     pid_list.append(pid_arr)
#     en_list.append(en_arr)

  0%|          | 0/14 [00:00<?, ?it/s]

Reading file step4_3_padded.h5
297401


  7%|▋         | 1/14 [02:10<28:14, 130.34s/it]

Reading file step4_7_padded.h5
297956


 14%|█▍        | 2/14 [03:13<22:00, 110.04s/it]

...

 86%|████████▌ | 12/14 [21:14<03:23, 101.61s/it]

Reading file step4_4_padded.h5
298635


 93%|█████████▎| 13/14 [22:14<01:28, 88.98s/it] 

Reading file step4_10_padded.h5
297171


100%|██████████| 14/14 [24:24<00:00, 101.42s/it]


In [20]:
data_list = []
pid_list = []
en_list = []

name = 'step4_10_padded.h5'
print("Reading file", name)
data = pd.read_hdf(dataset_dir + name)
num_tracks = data.trackster.max()
print(num_tracks)

tracksters = np.array([data.E.values, data.eta.values, data.phi.values]).T.reshape(-1, width, height, channels)

pid_vals = np.array([data.pid.values]).T.reshape(-1, width*height)
select = (np.max(pid_vals,axis=1)==0)
pid_vals=np.max(np.abs(pid_vals),axis=1)
pid_vals[select]=pid_vals[select]*-1.0
pid_vals = pid_vals.tolist()
pid_arr = [class_labels[x] for x in pid_vals]
en_vals = np.array([data.genE.values]).T.reshape(-1, width*height)
en_arr = np.max(en_vals,axis=1)

data_list.append(tracksters)
pid_list.append(pid_arr)
en_list.append(en_arr)

Reading file step4_10_padded.h5
297171


In [21]:
data_array = np.array([item for sublist in data_list for item in sublist])
print(data_array.shape)

(297171, 50, 10, 3)


In [22]:
data_new = np.transpose(data_array, (0,3,1,2))
print(data_new.shape)

(297171, 3, 50, 10)


In [23]:
pid_array = np.array([item for sublist in pid_list for item in sublist])
pid_array_cat = keras.utils.to_categorical(pid_array, num_classes=classes, dtype='float32')
print(pid_array_cat.shape)

(297171, 6)


In [24]:
en_array = np.array([item for sublist in en_list for item in sublist])
print(en_array.shape)

(297171,)


In [25]:
mean_en = np.mean(en_array)
std_en = np.std(en_array)
print('Mean Energy Value: {}'.format(mean_en))
print('Std Energy Value: {}'.format(std_en))

en_array_norm = (en_array - mean_en)/std_en

Mean Energy Value: 124.04123038025251
Std Energy Value: 149.31865148858864


In [16]:
# print('Creating model...')

# def full_model():

#     input_img = Input(shape=(width, height, channels), name='input')

#     conv = Conv2D(16, (5,1), activation='relu', padding='same', kernel_initializer='random_uniform', data_format='channels_last', name='conv1')(input_img)
#     conv = Conv2D(16, (3,3), activation='relu', padding='same', kernel_initializer='random_uniform', data_format='channels_last', name='conv2')(conv)
#     conv = Conv2D(16, (3,3), activation='relu', padding='same', kernel_initializer='random_uniform', data_format='channels_last', name='conv3')(conv)

#     flat = Flatten()(conv)

#     dense = Dense(512, activation='relu', kernel_initializer='random_uniform', name='dense1')(flat)
#     dense = Dense(128, activation='relu', kernel_initializer='random_uniform', name='dense2')(dense)

#     dense_id = Dense(64, activation='relu', kernel_initializer='random_uniform', name='dense_id1')(dense)
#     drop_id = Dropout(0.5)(dense_id)
#     dense_id = Dense(16, activation='relu', kernel_initializer='random_uniform', name='dense_id2')(drop_id)
#     drop_id = Dropout(0.5)(dense_id)

#     pid = Dense(classes, activation='softmax', kernel_initializer='random_uniform', name='pid_output')(drop_id)

#     dense_er = Dense(64, activation='relu', kernel_initializer='random_uniform', name='dense_er1')(dense)
#     dense_er = Dense(16, activation='relu', kernel_initializer='random_uniform', name='dense_er2')(dense_er)
    
#     enreg = Dense(1, name='enreg_output')(dense_er)

#     model = Model(inputs=input_img, outputs=[pid, enreg])

#     model.compile(loss={'pid_output': 'categorical_crossentropy', 'enreg_output': 'mse'}, loss_weights={'pid_output': 1.2, 'enreg_output': 1}, optimizer='adam', metrics={'pid_output': 'accuracy', 'enreg_output': 'mse'})
#     return model

# model = full_model()
# model.summary()

# print("Model created!")

Creating model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 50, 10, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 50, 10, 16)   256         input[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 50, 10, 16)   2320        conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, 50, 10, 16)   2320        conv2[0][0]                      
___________________________________________________________________________________________

In [26]:
print('Creating model...')

def full_model():

#     input_img = Input(shape=(width, height, channels), name='input')

#     conv = Conv2D(16, (5,1), activation='relu', padding='same', kernel_initializer='random_uniform', data_format='channels_last', name='conv1')(input_img)
#     conv = Conv2D(16, (3,3), activation='relu', padding='same', kernel_initializer='random_uniform', data_format='channels_last', name='conv2')(conv)
#     conv = Conv2D(16, (3,3), activation='relu', padding='same', kernel_initializer='random_uniform', data_format='channels_last', name='conv3')(conv)

    input_img = Input(shape=(channels, width, height), name='input')

    conv = Conv2D(16, (5,1), activation='relu', padding='same', data_format='channels_first', name='conv1')(input_img)
    conv = Conv2D(16, (3,3), activation='relu', padding='same', data_format='channels_first', name='conv2')(conv)
    conv = Conv2D(16, (3,3), activation='relu', padding='same', data_format='channels_first', name='conv3')(conv)


    flat = Reshape((-1,))(conv)

    dense = Dense(512, activation='relu', kernel_initializer='random_uniform', name='dense1')(flat)
    dense = Dense(128, activation='relu', kernel_initializer='random_uniform', name='dense2')(dense)

    dense_id = Dense(64, activation='relu', kernel_initializer='random_uniform', name='dense_id1')(dense)
#     drop_id = Dropout(0.5)(dense_id)
    dense_id = Dense(16, activation='relu', kernel_initializer='random_uniform', name='dense_id2')(dense_id)
#     drop_id = Dropout(0.5)(dense_id)

    pid = Dense(classes, activation='softmax', kernel_initializer='random_uniform', name='pid_output')(dense_id)

    dense_er = Dense(64, activation='relu', kernel_initializer='random_uniform', name='dense_er1')(dense)
    dense_er = Dense(16, activation='relu', kernel_initializer='random_uniform', name='dense_er2')(dense_er)
    
    enreg = Dense(1, name='enreg_output')(dense_er)

    model = Model(inputs=input_img, outputs=[pid, enreg])

    model.compile(loss={'pid_output': 'categorical_crossentropy', 'enreg_output': 'mse'}, loss_weights={'pid_output': 1.2, 'enreg_output': 1}, optimizer='adam', metrics={'pid_output': 'accuracy', 'enreg_output': 'mse'})
    return model

model = full_model()
model.summary()

print("Model created!")

Creating model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 3, 50, 10)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 50, 10)   256         input[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 16, 50, 10)   2320        conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, 16, 50, 10)   2320        conv2[0][0]                      
___________________________________________________________________________________________

In [27]:
history = model.fit(data_new, {'pid_output': pid_array_cat, 'enreg_output': en_array_norm}, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)], shuffle=True, verbose=1)
history_save = pd.DataFrame(history.history).to_hdf(save_dir + history_name + ".h5", "history", append=False)

Train on 267453 samples, validate on 29718 samples
Epoch 1/30
267453/267453 [==============================] - 28s 103us/step - loss: 1.0816 - pid_output_loss: 0.5747 - enreg_output_loss: 0.3920 - pid_output_acc: 0.7301 - enreg_output_mean_squared_error: 0.3920 - val_loss: 0.9728 - val_pid_output_loss: 0.4849 - val_enreg_output_loss: 0.3909 - val_pid_output_acc: 0.7466 - val_enreg_output_mean_squared_error: 0.3909
Epoch 2/30
267453/267453 [==============================] - 25s 95us/step - loss: 0.9451 - pid_output_loss: 0.4695 - enreg_output_loss: 0.3816 - pid_output_acc: 0.7570 - enreg_output_mean_squared_error: 0.3816 - val_loss: 0.9588 - val_pid_output_loss: 0.4728 - val_enreg_output_loss: 0.3915 - val_pid_output_acc: 0.7559 - val_enreg_output_mean_squared_error: 0.3915
Epoch 3/30
267453/267453 [==============================] - 26s 96us/step - loss: 0.9270 - pid_output_loss: 0.4555 - enreg_output_loss: 0.3803 - pid_output_acc: 0.7658 - enreg_output_mean_squared_error: 0.3803 - val_

In [28]:
# Save model and weights
model.save(save_dir + model_name + ".h5")
print('Saved trained model at %s ' % save_dir)

Saved trained model at /lustrehome/adipilato/ParticleID/PID-ER_v2/models/ 


In [29]:
# save the frozen model
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, save_dir, model_name + ".pbtxt", as_text=True)
tf.train.write_graph(frozen_graph, save_dir, model_name + ".pb", as_text=False)
print('Model saved')

INFO:tensorflow:Froze 279 variables.
INFO:tensorflow:Converted 279 variables to const ops.
Model saved


In [30]:
print("Done!")

Done!


In [15]:
unique, counts = np.unique(pid_array, return_counts=True)
print(unique)
print(counts)

[0 1 2 3 4 5]
[ 539297  538846   18173  488513  484131 1906340]
